Group 5: LI LINGYU, LIU YICHAO, WU JINGYAN, YANG QINGSHAN, YE FANGDA

# Read Data

In [ ]:
import pandas as pd
import config

In [ ]:
# Read cleaned csv file
chicago_clean = pd.read_csv(config.TRAIN_FILE_CLEAN)

# Feature Construction

In [ ]:
# Construct time features
chicago_clean['Date'] = pd.to_datetime(chicago_clean['Date'], format='%m/%d/%Y %I:%M:%S %p')

chicago_clean['Hour'] = chicago_clean['Date'].dt.hour
chicago_clean['Day'] = chicago_clean['Date'].dt.day
chicago_clean['Month'] = chicago_clean['Date'].dt.month
chicago_clean['DayOfWeek'] = chicago_clean['Date'].dt.dayofweek
chicago_clean['Year'] = chicago_clean['Date'].dt.year
chicago_clean['IsWeekend'] = chicago_clean['DayOfWeek'].isin([5,6]).astype(int)
chicago_clean['TimeOfDay'] = pd.cut(chicago_clean['Hour'], 
                                   bins=[0,6,12,18,24], 
                                   labels=['Night','Morning','Afternoon','Evening'])
chicago_clean['Season'] = pd.cut(chicago_clean['Month'],
                                bins=[0,3,6,9,12],
                                labels=['Winter','Spring','Summer','Fall'])

# Drop initial date column
chicago_clean = chicago_clean.drop('Date', axis=1)